In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [3]:
path = '../../KITTI/KITTI_gray/dataset/sequences/00/'

In [4]:
imT1_L = cv2.imread(path + "image_0/" + "000000.png", 0)
imT1_R = cv2.imread(path + "image_1/" + "000000.png", 0)

In [5]:
imT2_L = cv2.imread(path + "image_0/" + "000001.png", 0)
imT2_R = cv2.imread(path + "image_1/" + "000001.png", 0)

In [7]:
K1 =  np.array([[718.856 ,   0.    , 607.1928],
       [  0.    , 718.856 , 185.2157],
       [  0.    ,   0.    ,   1.    ]])

K2 =  np.array([[718.856 ,   0.    , 607.1928],
       [  0.    , 718.856 , 185.2157],
       [  0.    ,   0.    ,   1.    ]])

In [118]:
class StateBolts(object):
    
    """
    Acts as a state-storehouse which stores the left and right property of each
    """

    def __init__(self):
        self._left = None
        self._right = None
        
    @property
    def left(self):
        return self._left
    
    @left.setter
    def left(self, attribute):
        if attribute is None:
            raise ValueError("Attribute can not be None while setting the property")
        self._left = attribute

    @property
    def right(self):
        return self._right
    
    @right.setter
    def right(self, attribute):
        if attribute is None:
            raise ValueError("Attribute can not be None while setting the property")
        self._right = attribute


class VO_StateMachine():
    
    """
    The VO_state acts as a state-machine which stores the frames, descriptors, tracked features
    and matched features.  
    """
    
    def __init__(self, state_num=None, params=None):
    
        """
        Args:
            state_num : the number of the frame
        
        """
        self.state_num = state_num
        self.params = params
        
        #store information about frames 
        self.frames = StateBolts()
        
        #store information about the results of matched descriptors
        self.matchedPoints = StateBolts()
        self.keypoints = StateBolts()
        self.descriptors = StateBolts()
        self.inliers = StateBolts()
        
        
        #store triangulation results
        self.pts3D = None
        self.pts3D_Filter = None
        self.InliersFilter = StateBolts()

In [114]:
#from plot_utils import showMatchedFeatures
#from core.feature import getMatchingKeypoints

class DetectionEngine():
    
    def __init__(self, frames, params):
        
        self.frames = frames
        self.params = params

        self.matchedPoints = StateBolts()
        self.keypoints = StateBolts()
        self.descriptors = StateBolts()
        self.inliers = StateBolts()
                
    def matching_keypoints(self):
        if self.matchedPoints is None:
            self.process_matching_keypoints()
        return self.matchedPoints, self.keypoints, self.descriptors
    
    def process_matching_keypoints(self):
        self.matchedPoints, self.keypoints, self.descriptors = getMatchingKeypoints(self.frames.left, self.frames.right, self.params)
    
    def show_matching_keypoints(self):
        showMatchedFeatures(self.left_frame, self.right_frame)
        
    def filter_matching_inliers(self):
        
        for i in range(params.GeoComp.EM.num_trials):
            E, mask = cv2.findEssentialMatrix(self.matchedPoints.left, self.matchedPoints.right, params)
            mask = mask.astype(bool)
            ratio =  sum(mask)/len(mask.flatten())

            if ratio > params.EM.inilierRatio:
                print("Iterations : 5 point Algorithm : {}".format(i+1))
                print("Inlier Ratio :                   {}".format(ratio))
                break
            else:
                print("Failed to Calculate E, Iter :    {}".format(i))
                if i==params.GeoComp.EM.num_trials-1:
                    print("Maxinum interation in 5-point Algorithm reached")
                else:
                    print("Running Iterations Again. Iters left : {}".format(params.geocom.EM.num_trials-1))
                    
        self.inliers.left = self.matchedPoints.left(mask)
        self.inliers.right = self.matchedPoints.right(mask)
        
        return self.inliers.left, self.inliers.right

In [115]:
def get_matching_keypoint(left_frame, right_frame, params):
    pass
        
    
def showMatchedFeatures():
    pass

**Dataset here**

In [116]:
#define the dataset here

**Initialise the State**

In [ ]:
state_num = 0

left_frame, right_frame = dataset[state_num]

#initialise the state
prevState = VO_State(state_num)

#set frame state
prevState.frames.left, prevState.frames.right = left_frame, right_frame

**Find Matching Keypoints**

In [ ]:
detection_worker = DetectionEngine(prevState.frames, params)

In [ ]:
prevState.matchedPoints, prevState.keyPoints, prevState.descriptors = detection_worker.matching_keypoints()

if params.DEBUG.plotting:
    showMatchedFeatures(prevState.frames.left, prevState.frames, prevState.matchedPoints.left, prevState.matchedPoints.right)

In [ ]:
prevState.Inliers = detection_worker.filter_matching_inliers()

In [ ]:
prevState.pts3D, reproj_error = linear_triangulation(prevState.Inliers, dataset.ProjL, dataset.ProjR)

triangulation_filter_mask = filter_triangulated_points(prevState.pts3D)

prevState.InliersFilter.left = prevState.Inliers.left[triangulation_filter_mask]
prevState.InliersFilter.right = prevState.Inliers.right[triangulation_filter_mask]

In [ ]:
#if params.DEBUG.plotting:
showMatchedFeatures(prevState.frames.left, prevState.frames, prevState.InliersFilter.left, prevState.InliersFilter.left)